Get Data

In [41]:
import opendatasets as od 
import pandas as pd
import numpy as np
import os

if not os.path.exists('steam-games-dataset'):
    od.download( 
        "https://www.kaggle.com/datasets/fronkongames/steam-games-dataset/data") 
    
if not os.path.exists('steam-video-games'):
    od.download( 
        "https://www.kaggle.com/datasets/tamber/steam-video-games/data") 
    
if not os.path.exists('popularity-of-games-on-steam'):
    od.download( 
        "https://www.kaggle.com/datasets/michau96/popularity-of-games-on-steam") 

Load Data

In [72]:
user_data = pd.read_csv('steam-video-games\steam-200k.csv', encoding='ISO-8859-1', usecols=[0, 1, 2, 3], names=['UserID', 'Game', 'Behavior', 'Quantity'])
# Note: Behvior is either purchase or play. When behavior is purchase, quantity is always 1. When behavior is play, quantity is the number of hours

Find average hours played per game

In [70]:
hours_played = user_data[user_data['Behavior'] == 'play'].groupby('Game').agg(np.mean).sort_values(by='Quantity', ascending=False).drop('UserID', axis='columns')
hours_played.head(20)

,Quantity
Game,
Eastside Hockey Manager,1295.000000
Baldur's Gate II Enhanced Edition,475.255556
FIFA Manager 09,411.000000
Perpetuum,400.975000
Football Manager 2014,391.984615
Football Manager 2012,390.453165
Football Manager 2010,375.048571
Football Manager 2011,365.703226
Freaking Meatbags,331.000000


Normalize play quantity

In [141]:
normalized_user_data = user_data.copy()[user_data['Game'].isin(hours_played.index)]
normalized_user_data['Quantity'] = normalized_user_data.apply(lambda x: x['Quantity'] if x['Behavior'] == 'purchase' else x['Quantity'] / hours_played.loc[x['Game'], :]['Quantity'], axis=1)
normalized_user_data.head(10)

,UserID,Game,Behavior,Quantity
0,151603712,The Elder Scrolls V Skyrim,purchase,1.000000
1,151603712,The Elder Scrolls V Skyrim,play,2.607178
2,151603712,Fallout 4,purchase,1.000000
3,151603712,Fallout 4,play,1.349928
4,151603712,Spore,purchase,1.000000
5,151603712,Spore,play,0.506387
6,151603712,Fallout New Vegas,purchase,1.000000
7,151603712,Fallout New Vegas,play,0.234121
8,151603712,Left 4 Dead 2,purchase,1.000000
9,151603712,Left 4 Dead 2,play,0.212190


Combine rows

In [142]:
grouped_users = normalized_user_data.groupby(['UserID', 'Game']).aggregate({'Quantity': 'sum'})
grouped_users = grouped_users.reset_index(level='Game')
grouped_users.loc[151603712]

,Game,Quantity
UserID,,
151603712,Alan Wake,1.000000
151603712,BioShock,1.065645
151603712,BioShock 2,1.000000
151603712,BioShock Infinite,1.074707
151603712,Dead Island Epidemic,1.239623
151603712,Dota 2,1.002466
151603712,Dragon Age Origins - Ultimate Edition,1.030049
151603712,Eldevin,1.021525
151603712,Fallen Earth,1.000000


Pivot long to wide

In [150]:
#pivoted_users = grouped_users.pivot(index='UserID', columns='Game', values='Quantity')
pivoted_users = grouped_users.reset_index()
pivoted_users = pivoted_users.set_index(['UserID', 'Game'])
pivoted_users = pivoted_users.unstack(fill_value=0)
pivoted_users.head(50)

Quantity            \
Game    007 Legends 0RBITALIS   
UserID                          
5250            0.0       0.0   
76767           0.0       0.0   
86540           0.0       0.0   
103360          0.0       0.0   
144736          0.0       0.0   
181212          0.0       0.0   
229911          0.0       0.0   
298950          0.0       0.0   
299153          0.0       0.0   
381543          0.0       0.0   
547685          0.0       0.0   
554278          0.0       0.0   
561758          0.0       0.0   
577614          0.0       0.0   
604988          0.0       0.0   
622362          0.0       0.0   
635733          0.0       0.0   
683019          0.0       0.0   
714122          0.0       0.0   
748719          0.0       0.0   
835015          0.0       0.0   
861238          0.0       0.0   
871990          0.0       0.0   
937567          0.0       0.0   
948368          0.0       0.0   
975449          0.0       0.0   
976129          0.0       0.0   
983600          0.0       0.0   
994489          0.0       0.0   
1006880         0.0       0.0   
1024319         0.0       0.0   
1072465         0.0       0.0   
1129452         0.0       0.0   
1135775         0.0       0.0   
1268792         0.0       0.0   
1364546         0.0       0.0   
1423371         0.0       0.0   
1601069         0.0       0.0   
1601773         0.0       0.0   
1603625         0.0       0.0   
1612666         0.0       0.0   
1665511         0.0       0.0   
1699101         0.0       0.0   
1713646         0.0       0.0   
1834453         0.0       0.0   
1851828         0.0       0.0   
1936551         0.0       0.0   
1950243         0.0       0.0   
1971801         0.0       0.0   
2039434         0.0       0.0   

                                                                    \
Game    1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby)   
UserID                                                               
5250                                                   0.0           
76767                                                  0.0           
86540                                                  0.0           
103360                                                 0.0           
144736                                                 0.0           
181212                                                 0.0           
229911                                                 0.0           
298950                                                 0.0           
299153                                                 0.0           
381543                                                 0.0           
547685                                                 0.0           
554278                                                 0.0           
561758                                                 0.0           
577614                                                 0.0           
604988                                                 0.0           
622362                                                 0.0           
635733                                                 0.0           
683019                                                 0.0           
714122                                                 0.0           
748719                                                 0.0           
835015                                                 0.0           
861238                                                 0.0           
871990                                                 0.0           
937567                                                 0.0           
948368                                                 0.0           
975449                                                 0.0           
976129                                                 0.0           
983600                                                 0.0           
994489                                                 0.0           
1006880              

Feature Selection